## Sentinel-1 TOPS stack processor
The detailed algorithm for stack processing of TOPS data can be find here:

+ Fattahi, H., P. Agram, and M. Simons (2016), A Network-Based Enhanced Spectral Diversity Approach for TOPS Time-Series Analysis, IEEE Transactions on Geoscience and Remote Sensing, 55(2), 777-786, doi:[10.1109/TGRS.2016.2614925](https://ieeexplore.ieee.org/abstract/document/7637021).

-----------------------------------

The scripts provides support for Sentinel-1 TOPS stack processing. Currently supported workflows include a coregistered stack of SLC, interferograms, offsets, and coherence. 

#### Be sure [default] credentials in ~/.aws/credentials are valid and the ~/.netrc file has been changed to include valid credentials for urs.earthdata.nasa.gov.

In [ ]:
import os
import json
from math import floor, ceil
import json
import re
import osaka
import osaka.main
from builtins import str
import os, sys, re, json, logging, traceback, requests, argparse
from datetime import datetime
from pprint import pformat
from requests.packages.urllib3.exceptions import (InsecureRequestWarning,
                                                  InsecurePlatformWarning)
try: from html.parser import HTMLParser
except: from html.parser import HTMLParser
        
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
requests.packages.urllib3.disable_warnings(InsecurePlatformWarning)
PROCESSING_START=datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
print(PROCESSING_START)

PGE_BASE=os.getcwd()
print(PGE_BASE)

In [ ]:
SLC_RE = re.compile(r'(?P<mission>S1\w)_IW_SLC__.*?' +
                    r'_(?P<start_year>\d{4})(?P<start_month>\d{2})(?P<start_day>\d{2})' +
                    r'T(?P<start_hour>\d{2})(?P<start_min>\d{2})(?P<start_sec>\d{2})' +
                    r'_(?P<end_year>\d{4})(?P<end_month>\d{2})(?P<end_day>\d{2})' +
                    r'T(?P<end_hour>\d{2})(?P<end_min>\d{2})(?P<end_sec>\d{2})_.*$')
ISCE_HOME="/opt/isce2/isce"

QC_SERVER = 'https://qc.sentinel1.eo.esa.int/'
DATA_SERVER = 'http://aux.sentinel1.eo.esa.int/'

ORBITMAP = [('precise','aux_poeorb', 100),
            ('restituted','aux_resorb', 100)]

OPER_RE = re.compile(r'S1\w_OPER_AUX_(?P<type>\w+)_OPOD_(?P<yr>\d{4})(?P<mo>\d{2})(?P<dy>\d{2})')

In [ ]:
from typing import List
ISCE_HOME="/opt/isce2/isce"
min_lat = 34.6002832
max_lat = 34.6502392
min_lon = -79.0801608
max_lon = -78.9705888
master_date = ""
localize_slcs: List[str] = ["S1A_IW_SLC__1SDV_20150315T231319_20150315T231349_005049_006569_0664",
                      "S1A_IW_SLC__1SDV_20150818T231326_20150818T231356_007324_00A0E0_93D5",
                      "S1A_IW_SLC__1SDV_20150830T231332_20150830T231402_007499_00A5A9_02B3",
                      "S1A_IW_SLC__1SDV_20160414T231321_20160414T231350_010824_01030B_F02B",
                      "S1A_IW_SLC__1SDV_20160414T231348_20160414T231416_010824_01030B_9FA9"
                     ]

In [ ]:
ctx = {}
ctx["min_lat"] = min_lat
ctx["max_lat"] = max_lat
ctx["min_lon"] = min_lon
ctx["max_lon"] = max_lon
ctx["master_date"]=""
ctx["localize_slcs"]= localize_slcs

print(json.dumps(ctx, indent=4))
wd = os.getcwd()
global runtime_dict
runtime_dict = {}

In [ ]:
with open('_stdout.txt', 'w') as f:
    f.write("Output File")
with open('_context.json', 'w') as outfile:
    json.dump(ctx, outfile)

In [ ]:
class MyHTMLParser(HTMLParser):

    def __init__(self):
        HTMLParser.__init__(self)
        self.fileList = []
        self.pages = 0
        self.in_td = False
        self.in_a = False
        self.in_ul = False

    def handle_starttag(self, tag, attrs):
        if tag == 'td':
            self.in_td = True
        elif tag == 'a' and self.in_td:
            self.in_a = True
        elif tag == 'ul':
            for k,v in attrs:
                if k == 'class' and v.startswith('pagination'):
                    self.in_ul = True
        elif tag == 'li' and self.in_ul:
            self.pages += 1

    def handle_data(self,data):
        if self.in_td and self.in_a:
            if OPER_RE.search(data):
                self.fileList.append(data.strip())

    def handle_endtag(self, tag):
        if tag == 'td':
            self.in_td = False
            self.in_a = False
        elif tag == 'a' and self.in_td:
            self.in_a = False
        elif tag == 'ul' and self.in_ul:
            self.in_ul = False
        elif tag == 'html':
            if self.pages == 0:
                self.pages = 1
            else:
                # decrement page back and page forward list items
                self.pages -= 2

In [ ]:
def session_get(session, url):
    return session.get(url, verify=False)

def get_download_orbit_dict(download_orbit_dict, slc_date, mission_type):
    

    url = "https://qc.sentinel1.eo.esa.int/aux_poeorb/?validity_start={}&sentinel1__mission={}".format(slc_date, mission_type)
    session = requests.Session()
    r = session_get(session, url)
    r.raise_for_status()
    parser = MyHTMLParser()
    parser.feed(r.text)

    for res in parser.fileList:
        #id = "%s-%s" % (os.path.splitext(res)[0], dataset_version)
        match = OPER_RE.search(res)
        if not match:
            raise RuntimeError("Failed to parse orbit: {}".format(res))
        download_orbit_dict[res] = os.path.join(DATA_SERVER, "/".join(match.groups()), "{}.EOF".format(res))
        #yield id, results[id]
        
    #print(results)
    
    return download_orbit_dict

In [ ]:
def get_orbit_files():
    from datetime import datetime
    import json
    import os
    import osaka
    import osaka.main
    
    orbit_dict = {}

    orbit_dates = []
    
    for slc in ctx["localize_slcs"]:
        match = SLC_RE.search(slc)
        if not match:
            raise RuntimeError("Failed to recognize SLC ID %s." %slc)
        mission = match.group('mission')
        day_dt_str = "{}-{}-{}".format(match.group('start_year'), 
                                       match.group('start_month'), match.group('start_day'))
        
        if day_dt_str not in orbit_dates:
            orbit_dates.append(day_dt_str)
            orbit_dict = get_download_orbit_dict(orbit_dict, day_dt_str, mission)
            directory = os.path.join(wd, "orbits")
            if not os.path.exists(directory):
                os.makedirs(directory)
            #directory = os.path.join(wd, "orbits")
            for k, v in orbit_dict.items():
                osaka.main.get(v, directory)
         
    print("orbit_dict : %s " %json.dumps(orbit_dict, indent=4))

In [ ]:
def get_current_time():
    import datetime
    return datetime.datetime.now()

def download_slc(slc_id, path):
    url = "https://datapool.asf.alaska.edu/SLC/SA/{}.zip".format(slc_id)
    print("Downloading {} : {}".format(slc_id, url))
    
    if not os.path.exists(path):
        os.makedirs(path)
    osaka.main.get(url, path)

def run_cmd_output(cmd):
    from subprocess import check_output, CalledProcessError
    cmd_line = " ".join(cmd)
    print("Calling: {}".format(cmd_line))
    output = check_output(cmd_line, shell=True)
    return output

def run_cmd(cmd):
    !source /opt/isce2/isce_env.sh
    !export PATH=/opt/conda/bin/:/opt/isce2/src/isce2/contrib/stack/topsStack/:${PATH}
    !export PYTHONPATH=/opt/isce2/src/isce2/contrib/stack/topsStack/:${PYTHONPATH}

    import subprocess
    from subprocess import check_call, CalledProcessError
    import sys
    cmd_line = " ".join(cmd)
    print("Calling : {}".format(cmd_line))
    p = subprocess.Popen(cmd_line, shell=True,stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
    while True: 
        line = p.stdout.readline()
        if not line:
            break
        print(line.strip())
        sys.stdout.flush()
        
def get_minimum_bounding_rectangle():
    from math import floor, ceil
    cwd = os.getcwd()
    slc_ids = [x for x in os.listdir('.') if os.path.isdir(x) and '_SLC__' in x]

    all_lats = []
    all_lons = []
    for slc in slc_ids:
        slc_met_json = slc + '.met.json'

        with open(os.path.join(cwd, slc, slc_met_json), 'r') as f:
            data = json.load(f)
            bbox = data['bbox']
            for coord in bbox:
                all_lats.append(coord[0])
                all_lons.append(coord[1])

    min_lat = min(all_lats) + 0.2
    max_lat = max(all_lats) - 0.1
    min_lon = min(all_lons)
    max_lon = max(all_lons)

    min_lat_lo = floor(min_lat)
    max_lat_hi = ceil(max_lat)
    min_lon_lo = floor(min_lon)
    max_lon_hi = ceil(max_lon)

    return min_lat, max_lat, min_lon, max_lon, min_lat_lo, max_lat_hi, min_lon_lo, max_lon_hi

def get_user_input_bbox(ctx_file):
    """
    :param ctx_file: dictionary from cxt file
    :return: void
    """
    from math import floor, ceil
    min_lat = ctx_file['min_lat']
    max_lat = ctx_file['max_lat']
    min_lon = ctx_file['min_lon']
    max_lon = ctx_file['max_lon']

    min_lat_lo = floor(min_lat)
    max_lat_hi = ceil(max_lat)
    min_lon_lo = floor(min_lon)
    max_lon_hi = ceil(max_lon)

    return min_lat, max_lat, min_lon, max_lon, min_lat_lo, max_lat_hi, min_lon_lo, max_lon_hi

def get_master_date(ctx):
    master_date = ctx.get('master_date', "")
    return master_date

def get_bbox(ctx):
    # min_lat, max_lat, min_lon, max_lon = ctx['region_of_interest']

    if ctx['min_lat'] != "" or ctx['max_lat'] != "" or ctx['min_lon'] != "" or ctx['max_lon'] != "":
        # if any values are present in _context.json we can assume user put them in manually
        bbox_data = get_user_input_bbox(ctx)
    else:
        # if user did not define ANY lat lons
        bbox_data = get_minimum_bounding_rectangle()

    return bbox_data

def download_dem():
    dem_cmd = [
        "{}/applications/dem.py".format(ISCE_HOME), "-a",
        "stitch", "-b", "{} {} {} {}".format(MINLAT_LO, MAXLAT_HI, MINLON_LO, MAXLON_HI),
        "-r", "-s", "1", "-f", "-c", "|", "tee", "dem.txt"
        #"-n", dem_user, "-w", dem_pass,"-u", dem_url
    ]
    run_cmd(dem_cmd)

In [ ]:
import os
import sys
import shutil
import json
import re

from datetime import datetime
from osgeo import ogr, osr

def get_union_polygon_from_bbox(env):
    coords = [
        [ env[3], env[0] ],
        [ env[3], env[1] ],
        [ env[2], env[1] ],
        [ env[2], env[0] ],
    ]
    return {
        "type": "Polygon",
        "coordinates": [ coords ]
    }

# copied from stitch_ifgs.get_union_polygon()
def get_union_polygon(ds_files):
    """
    Get GeoJSON polygon of union of IFGs.
    :param ds_files: list of .dataset.json files, which have the 'location' key
    :return: geojson of merged bbox
    """

    geom_union = None
    for ds_file in ds_files:
        f = open(ds_file)
        ds = json.load(f)
        geom = ogr.CreateGeometryFromJson(json.dumps(ds['location'], indent=2, sort_keys=True))
        if geom_union is None:
            geom_union = geom
        else:
            geom_union = geom_union.Union(geom)
    return json.loads(geom_union.ExportToJson()), geom_union.GetEnvelope()


def get_dataset_met_json_files(cxt):
    """
    returns 2 lists: file paths for dataset.json files and met.json files
    :param cxt: json from _context.json
    :return: list[str], list[str]
    """
    pwd = os.getcwd()
    localize_urls = cxt['localize_slcs']

    met_files, ds_files = [], []
    for localize_url in localize_urls:
        local_path = localize_url['local_path']
        slc_id = local_path.split('/')[0]
        slc_path = os.path.join(pwd, slc_id, slc_id)

        ds_files.append(slc_path + '.dataset.json')
        met_files.append(slc_path + '.met.json')
    return ds_files, met_files


def get_scenes(cxt):
    """
    gets all SLC scenes for the stack
    :param cxt: contents for _context.json
    :return: list of scenes
    """
    localize_urls = cxt['localize_slcs']
    all_scenes = set()
    for localize_url in localize_urls:
        local_path = localize_url['local_path']
        slc_id = local_path.split('/')[0]
        all_scenes.add(slc_id)
    return sorted(list(all_scenes))


def get_min_max_timestamps(scenes_ls):
    """
    returns the min timestamp and max timestamp of the stack
    :param scenes_ls: list[str] all slc scenes in stack
    :return: (str, str) 2 timestamp strings, ex. 20190518T161611
    """
    timestamps = set()

    regex_pattern = r'(\d{8}T\d{6}).(\d{8}T\d{6})'
    for scene in scenes_ls:
        matches = re.search(regex_pattern, scene)
        if not matches:
            raise Exception("regex %s was unable to match with SLC id %s" % (regex_pattern, scene))

        slc_timestamps = (matches.group(1), matches.group(2))
        timestamps = timestamps.union(slc_timestamps)

    min_timestamp = min(timestamps)
    max_timestamp = max(timestamps)
    return min_timestamp.replace('T', ''), max_timestamp.replace('T', '')

def get_min_max_times(scenes_ls):
    """
    returns the min timestamp and max timestamp of the stack
    :param scenes_ls: list[str] all slc scenes in stack
    :return: (str, str) 2 timestamp strings, ex. 20190518T161611
    """
    timestamps = set()

    regex_pattern = r'(\d{8}T\d{6}).(\d{8}T\d{6})'
    for scene in scenes_ls:
        matches = re.search(regex_pattern, scene)
        if not matches:
            raise Exception("regex %s was unable to match with SLC id %s" % (regex_pattern, scene))

        slc_timestamps = (matches.group(1), matches.group(2))
        timestamps = timestamps.union(slc_timestamps)

    min_timestamp = min(timestamps)
    max_timestamp = max(timestamps)
    return min_timestamp, max_timestamp


def create_list_from_keys_json_file(json_files, *args):
    """
    gets all key values in each .json file and returns a sorted array of values
    :param json_files: list[str]
    :return: list[]
    """
    values = set()
    for json_file in json_files:
        if os.path.isfile(json_file): 
            f = open(json_file)
            data = json.load(f)
            for arg in args:
                value = data[arg]
                values.add(value)
    return sorted(list(values))


def camelcase_to_underscore(name):
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()


def get_key_and_convert_to_underscore(json_file_paths, key):
    """
    read through all the json files in file paths, get the first occurrence of key and convert it to underscore
    :param json_file_paths: list[str]
    :param key: str
    :return: key and value
    """
    for json_file in json_file_paths:
        if os.path.isfile(json_file): 
            f = open(json_file)
            data = json.load(f)
            if key in data.keys():
                underscore_key = camelcase_to_underscore(key)
                return underscore_key, data[key]
    return None, None


def generate_dataset_json_data(ctx, version):
    """
    :param cxt: _context.json file
    :param dataset_json_files: list[str] all file paths of SLC's .dataset.json files
    :param version: str: version, ex. v1.0
    :return: dict
    """
    dataset_json_data = dict()
    dataset_json_data['version'] = version

    
    try:      
        dataset_json_data['starttime'], dataset_json_data['endtime']  = get_min_max_times(ctx["localize_slcs"])      
    except Exception as err:
        print(str(err))
    try:
        dataset_json_data['location'] = get_union_polygon_from_bbox(bbox.split(' '))
    except Exception as err:
        print(str(err))
    return dataset_json_data


def generate_met_json_data(ctx, bbox, version):
    """
    :param cxt: _context.json file
    :param met_json_file_paths: list[str] all file paths of SLC's .met.json files
    :param dataset_json_files: list[str] all file paths of SLC's .dataset.json files
    :param version: str: version, ex. v1.0
    :return: dict
    """
    met_json_data = {
        'processing_start': PROCESSING_START,
        'processing_stop': datetime.now().strftime('%Y-%m-%dT%H:%M:%S'),
        'version': version
    }

    # generating bbox
    geojson = get_union_polygon_from_bbox(bbox)
    coordinates = geojson['coordinates'][0]
    for coordinate in coordinates:
        coordinate[0], coordinate[1] = coordinate[1], coordinate[0]
    
    met_json_data['bbox'] = bbox
    
    # list of SLC scenes
   
    met_json_data['scenes'] = ctx["localize_slcs"]
    met_json_data['scene_count'] = len(ctx["localize_slcs"])

    # getting timestamps

    met_json_data['sensing_start'], met_json_data['sensing_stop']  = get_min_max_times(ctx["localize_slcs"])

    # additional information
    met_json_data['dataset_type'] = 'stack'

    return met_json_data

In [ ]:
def read_context():
    with open('_context.json', 'r') as f:
        cxt = json.load(f)
        return cxt


def create_dataset(bbox):
    VERSION = 'v1.0'
    DATASET_NAMING_TEMPLATE = 'coregistered_slcs-{min_timestamp}-{max_timestamp}'
    PWD = os.getcwd()

    # creating list of all SLC .dataset.json and .met.json files
    ctx = read_context()
    #dataset_json_files, met_json_files = get_dataset_met_json_files(context_json)

    # getting list of SLC scenes and extracting min max timestamp
    slc_scenes = ctx['localize_slcs']
    min_timestamp, max_timestamp = get_min_max_timestamps(slc_scenes)

    # creatin dataset directory
    dataset_name = DATASET_NAMING_TEMPLATE.format(min_timestamp=min_timestamp, max_timestamp=max_timestamp)
    if not os.path.exists(dataset_name):
        os.mkdir(dataset_name)

    # move merged/ master/ slaves/ directory to dataset directory
    move_directories = ['merged', 'reference', 'secondarys']
    for directory in move_directories:
        shutil.move(directory, dataset_name)

    # move _stdout.txt log file to dataset
    shutil.copyfile('_stdout.txt', os.path.join(dataset_name, '_stdout.txt'))

    # generate .dataset.json data
    dataset_json_data = {}
    try:
        dataset_json_data = generate_dataset_json_data(ctx, VERSION)
    except Exception as err:
        print(str(err))
    dataset_json_data['label'] = dataset_name
    print(json.dumps(dataset_json_data, indent=2))

    # generate .met.json data
    met_json_data = generate_met_json_data(ctx, bbox, VERSION)
    print(json.dumps(met_json_data, indent=2))

    # writing .dataset.json to file
    dataset_json_filename = os.path.join(PWD, dataset_name, dataset_name + '.dataset.json')
    with open(dataset_json_filename, 'w') as f:
        json.dump(dataset_json_data, f, indent=2)

    # writing .met.json to file
    met_json_filename = os.path.join(PWD, dataset_name, dataset_name + '.met.json')
    with open(met_json_filename, 'w') as f:
        json.dump(met_json_data, f, indent=2)

In [ ]:
MINLAT, MAXLAT, MINLON, MAXLON, MINLAT_LO, MAXLAT_HI, MINLON_LO, MAXLON_HI =get_bbox(ctx)
print("{} {} {} {} {} {} {} {}".format(MINLAT, MAXLAT, MINLON, MAXLON, MINLAT_LO, MAXLAT_HI, MINLON_LO, MAXLON_HI))

bbox = "{} {} {} {}".format(MINLAT, MAXLAT, MINLON, MAXLON)


#### Download Sentinel-1 data SLC ####


In [ ]:
def download_slcs(path):
    
    for slc in ctx["localize_slcs"]:
        download_slc(slc, path)
        
path = "zip"
download_slcs(path)

#### 3. Download Orbit Files based onn SLC ####

In [ ]:
get_orbit_files()

In [ ]:
download_dem()

In [ ]:
import os
cwd = os.getcwd()
print("cwd : {}".format(cwd))
if os.path.exists("dem.txt"):
    cmd = ["awk", "'/wgs84/ {print $NF;exit}'", "dem.txt"]
    WGS84 = run_cmd_output(cmd).decode("utf-8").strip()
    wgs84_file = os.path.join(cwd, WGS84)
    print("WGS84 : a{}b".format(wgs84_file))
    if os.path.exists(wgs84_file):
        print("Found wgs84 file: {}".format(wgs84_file))
        fix_cmd = ["{}/applications/fixImageXml.py".format(ISCE_HOME), "--full", "-i", "{}".format(wgs84_file) ]
        run_cmd(fix_cmd) 
    else:
        print("NO WGS84 FILE FOUND : {}".format(wgs84_file))

#### AUX_CAL file download ####

The following calibration auxliary (AUX_CAL) file is used for **antenna pattern correction** to compensate the range phase offset of SAFE products with **IPF verison 002.36** (mainly for images acquired before March 2015). If all your SAFE products are from another IPF version, then no AUX files are needed. Check [ESA document](https://earth.esa.int/documents/247904/1653440/Sentinel-1-IPF_EAP_Phase_correction) for details. 

Run the command below to download the AUX_CAL file once and store it somewhere (_i.e._ ~/aux/aux_cal) so that you can use it all the time, for `stackSentinel.py -a` or `auxiliary data directory` in `topsApp.py`.

```
wget https://qc.sentinel1.eo.esa.int/product/S1A/AUX_CAL/20140908T000000/S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ
tar zxvf S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ
rm S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ
```

In [ ]:
%%bash
mkdir -p ./AuxDir
wget https://qc.sentinel1.eo.esa.int/product/S1A/AUX_CAL/20140908T000000/S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ
tar zxvf S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ --directory ./AuxDir
rm S1A_AUX_CAL_V20140908T000000_G20190626T100201.SAFE.TGZ


The scripts provides support for Sentinel-1 TOPS stack processing. Currently supported workflows include a coregistered stack of SLC, interferograms, offsets, and coherence. 

`stackSentinel.py` generates all configuration and run files required to be executed on a stack of Sentinel-1 TOPS data. When stackSentinel.py is executed for a given workflow (-W option) a **configs** and **run_files** folder is generated. No processing is performed at this stage. Within the run_files folder different run\_#\_description files are contained which are to be executed as shell scripts in the run number order. Each of these run scripts call specific configure files contained in the “configs” folder which call ISCE in a modular fashion. The configure and run files will change depending on the selected workflow. To make run_# files executable, change the file permission accordingly (e.g., `chmod +x run_01_unpack_slc`).

```bash
stackSentinel.py -H     #To see workflow examples,
stackSentinel.py -h     #To get an overview of all the configurable parameters
```

Required parameters of stackSentinel.py include:

```cfg
-s SLC_DIRNAME          #A folder with downloaded Sentinel-1 SLC’s. 
-o ORBIT_DIRNAME        #A folder containing the Sentinel-1 orbits. Missing orbit files will be downloaded automatically
-a AUX_DIRNAME          #A folder containing the Sentinel-1 Auxiliary files
-d DEM_FILENAME         #A DEM (Digital Elevation Model) referenced to wgs84
```

In the following, different workflow examples are provided. Note that stackSentinel.py only generates the run and configure files. To perform the actual processing, the user will need to execute each run file in their numbered order.

In all workflows, coregistration (-C option) can be done using only geometry (set option = geometry) or with geometry plus refined azimuth offsets through NESD (set option = NESD) approach, the latter being the default. For the NESD coregistrstion the user can control the ESD coherence threshold (-e option) and the number of overlap interferograms (-O) to be used in NESD estimation.


In [ ]:
master_date=get_master_date(ctx)
print("master_date : {}".format(master_date))

if master_date:
    print("MASTER_DATE exists:".format(master_date) )   
    cmd = [
        "/opt/isce2/src/isce2/contrib/stack/topsStack/stackSentinel.py",  "-s", "zip/", "-d", "{}".format(wgs84_file), "-a", "AuxDir/", "-m", "{}".format(master_date), "-o", "orbits", 
        "-b", "\"{} {} {} {}\"".format(MINLAT, MAXLAT, MINLON, MAXLON), 
        "-W", "slc", "-C", "geometry"
    ]
else:
    print("MASTER_DATE DOES NOT EXIST")          
    cmd = [
        "/opt/isce2/src/isce2/contrib/stack/topsStack/stackSentinel.py",  "-s", "zip/", "-d", "{}".format(wgs84_file), "-a", "AuxDir/", "-o", "orbits", 
        "-b", "\"{} {} {} {}\"".format(MINLAT, MAXLAT, MINLON, MAXLON), 
        "-W", "slc", "-C", "geometry"
    ]
run_cmd(cmd)

In [ ]:
cmd = ["~/topsstack-hamsar/topsStack/stackSlcDn_run2.5.sh", "{} {} {} {}".format(MINLAT, MAXLAT, MINLON, MAXLON)]
run_cmd(cmd)

**run_01_unpack_slc_topo_reference:**

Includes commands to unpack Sentinel-1 TOPS SLCs using ISCE readers. For older SLCs which need antenna elevation pattern correction, the file is extracted and written to disk. For newer version of SLCs which don’t need the elevation antenna pattern correction, only a gdal virtual “vrt” file (and isce xml file) is generated. The “.vrt” file points to the Sentinel SLC file and reads them whenever required during the processing. If a user wants to write the “.vrt” SLC file to disk, it can be done easily using gdal_translate (e.g. gdal_translate –of ENVI File.vrt File.slc). 
The “run_01_unpack_slc_topo_reference” also includes a command that refers to the config file of the stack reference, which includes configuration for running topo for the stack reference. Note that in the pair-wise processing strategy one should run topo (mapping from range-Doppler to geo coordinate) for all pairs. However, with stackSentinel, topo needs to be run only one time for the reference in the stack. 


In [ ]:
runtime_dict["s1_start"]=get_current_time()

In [ ]:
%%bash
echo "## STEP 1 ##"

start=`date +%s`
echo "sh run_files/run_01_unpack_topo_reference"
sh run_files/run_01_unpack_topo_reference
end=`date +%s`
runtime1=$((end-start))
echo $runtime1

In [ ]:
runtime_dict["s1_stop"]=get_current_time()
runtime_dict["s1_runtime"]=runtime_dict["s1_stop"]-runtime_dict["s1_start"]
print(runtime_dict)

**run_02_unpack_secondary_slc:**

In [ ]:
%%bash
echo "## STEP 2 ##"

start=`date +%s`
Num=`cat run_files/run_02_unpack_secondary_slc | wc | awk '{print $1}'`
echo $Num
echo "cat run_files/run_02_unpack_secondary_slc | parallel -j2 --eta --load 50%"
cat run_files/run_02_unpack_secondary_slc | parallel -j2 --eta --load 50%
end=`date +%s`

runtime2=$((end-start))
echo runtime2

In [ ]:
runtime_dict["s2_stop"]=get_current_time()
runtime_dict["s2_runtime"]=runtime_dict["s2_stop"]-runtime_dict["s1_stop"]
print(runtime_dict)

#### run_02.5_slc_noise_calibration ###
STEP 2.5 run radiometric and thermal noise calibration

In [ ]:
%%bash
echo "## STEP 2.5 ##"

start=`date +%s`
echo "cat run_files/run_02.5_slc_noise_calibration | parallel -j2 --eta --load 50%"
cat run_files/run_02.5_slc_noise_calibration | parallel -j2 --eta --load 50%
end=`date +%s`

runtime2x5=$((end-start))
echo $runtime2x5

In [ ]:
runtime_dict["s2.5_stop"]=get_current_time()
runtime_dict["s2.5_runtime"]=runtime_dict["s2.5_stop"]-runtime_dict["s2_stop"]
print(runtime_dict)

**run_03_average_baseline:**

Computes average baseline for the stack. These baselines are not used for processing anywhere. They are only an approximation and can be used for plotting purposes.

In [ ]:
%%bash
echo "## STEP 3 ##"

start=`date +%s`
echo "cat run_files/run_03_average_baseline | parallel -j2 --eta --load 50%"
cat run_files/run_03_average_baseline | parallel -j2 --eta --load 50%
end=`date +%s`
runtime3=$((end-start))
echo $runtime3

In [ ]:
runtime_dict["s3_stop"]=get_current_time()
runtime_dict["s3_runtime"]=runtime_dict["s3_stop"]-runtime_dict["s2.5_stop"]
print(runtime_dict)

**STEP 4 : run_04_fullBurst_geo2rdr:**



In [ ]:
%%bash
echo "## STEP 4 ##"

start=`date +%s`

echo "cat run_files/run_04_fullBurst_geo2rdr  | parallel -j2 --eta --load 50%"
cat run_files/run_04_fullBurst_geo2rdr  | parallel -j2 --eta --load 50%
end=`date +%s`
runtime4=$((end-start))
echo $runtime4

In [ ]:
runtime_dict["s4_stop"]=get_current_time()
runtime_dict["s4_runtime"]=runtime_dict["s4_stop"]-runtime_dict["s3_stop"]
print(runtime_dict)

**STEP 5 : run_05_fullBurst_resample:**

In [ ]:
%%bash
echo "## STEP 5 ##"

start=`date +%s`
echo "cat run_files/run_05_fullBurst_resample  | parallel -j2 --eta --load 50%"
cat run_files/run_05_fullBurst_resample  | parallel -j2 --eta --load 50%
end=`date +%s`
runtime5=$((end-start))
echo $runtime5

In [ ]:
runtime_dict["s5_stop"]=get_current_time()
runtime_dict["s5_runtime"]=runtime_dict["s5_stop"]-runtime_dict["s4_stop"]
print(runtime_dict)

**STEP 6 : run_06_timeseries_misreg:**

A time-series of azimuth and range misregistration is estimated with respect to the stack reference. The time-series is a least squares esatimation from the pair misregistration from the previous step.


In [ ]:
%%bash
echo "## STEP 6 ##"

start=`date +%s`
echo "sh run_files/run_06_extract_stack_valid_region"
sh run_files/run_06_extract_stack_valid_region
end=`date +%s`
runtime6=$((end-start))
echo $runtime6

In [ ]:
runtime_dict["s6_stop"]=get_current_time()
runtime_dict["s6_runtime"]=runtime_dict["s6_stop"]-runtime_dict["s5_stop"]
print(runtime_dict)

**run_07_geo2rdr_resample:**

Using orbit and DEM, geometrical offsets among all secondary SLCs and the stack reference is computed. The geometrical offsets, together with the misregistration time-series (from previous step) are used for precise coregistration of each burst SLC. 

In [ ]:
%%bash
echo "## STEP 7 ##"

start=`date +%s`

FILE=run_files/run_07_merge
if test -f "$FILE"; then
    echo "cat run_files/run_07_merge  | parallel -j2 --eta --load 50%"
    cat run_files/run_07_merge | parallel -j2 --eta --load 50%
else
    echo "cat run_files/run_07_merge_reference_secondary_slc  | parallel -j2 --eta --load 50%"
    cat run_files/run_07_merge_reference_secondary_slc | parallel -j2 --eta --load 50%
fi

end=`date +%s`
runtime7=$((end-start))
echo $runtime7

In [ ]:
runtime_dict["s7_stop"]=get_current_time()
runtime_dict["s7_runtime"]=runtime_dict["s7_stop"]-runtime_dict["s6_stop"]
print(runtime_dict)

In [ ]:
print("time before creating dataset : {}".format(get_current_time()))
create_dataset(bbox)
print("time after creating dataset : {}".format(get_current_time()))